<a href="https://colab.research.google.com/github/Lingotech-Davis/LowResourceLLMTuning/blob/main/practice_notebooks/tamil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Whisper Demo 1

In [6]:
!pip install --upgrade --no-deps --force-reinstall git+https://github.com/openai/whisper.git
!pip install --upgrade datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio
!pip install -q datacollective datasets transformers soundfile
!pip install -U transformers datasets accelerate


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-6k6p7rps
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-6k6p7rps
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=08c757770cca610a55833ee857016d3bf63e4e1c50b5d187e28e870f57d620cf
  Stored in directory: /tmp/pip-ephem-wheel-cache-mva9wjzh/wheels/c3/03/25/5e0ba78bc27a3a089f137c9f1d92fdfce16d06996c071a016c
Successfully built openai-whisper
  Attempting uninstall: openai-whisper
    Found existing installation: openai-whisper 20250625
    Uninstalling openai-whisper-20250625:
      Successfully uninstalled openai-whisper-20250625


In [7]:
import whisper
from huggingface_hub import notebook_login
notebook_login()

from google.colab import userdata
api_key = userdata.get('MDC_API_KEY')

import os
os.environ["MDC_API_KEY"] = api_key


load dataset to the disk (5 minutes)

In [8]:
from datacollective import save_dataset_to_disk

dataset_id = "cmj8u3pv200qpnxxbgpfrn7la"

local_path = save_dataset_to_disk(dataset_id)
print("Dataset saved to:", local_path)

File already exists. Skipping download: `/root/.mozdata/datasets/mcv-scripted-ta-v24.0.tar.gz`
Dataset saved to: /root/.mozdata/datasets/mcv-scripted-ta-v24.0.tar.gz


In [9]:
!gunzip -k /root/.mozdata/datasets/mcv-scripted-ta-v24.0.tar.gz

gzip: /root/.mozdata/datasets/mcv-scripted-ta-v24.0.tar already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [10]:
!ls -l /root/.mozdata/datasets

total 18509512
-rw-r--r-- 1 root root 9757476864 Dec 26 08:29 mcv-scripted-ta-v24.0.tar
-rw-r--r-- 1 root root 9196249656 Dec 26 08:29 mcv-scripted-ta-v24.0.tar.gz


In [11]:
!tar -xvf /root/.mozdata/datasets/mcv-scripted-ta-v24.0.tar

Streaming output truncated to the last 5000 lines.
cv-corpus-24.0-2025-12-05/ta/clips/common_voice_ta_40910506.mp3
cv-corpus-24.0-2025-12-05/ta/clips/common_voice_ta_40910507.mp3
cv-corpus-24.0-2025-12-05/ta/clips/common_voice_ta_40910529.mp3
cv-corpus-24.0-2025-12-05/ta/clips/common_voice_ta_40910530.mp3
cv-corpus-24.0-2025-12-05/ta/clips/common_voice_ta_40910531.mp3
cv-corpus-24.0-2025-12-05/ta/clips/common_voice_ta_40910532.mp3
cv-corpus-24.0-2025-12-05/ta/clips/common_voice_ta_40910533.mp3
cv-corpus-24.0-2025-12-05/ta/clips/common_voice_ta_40910544.mp3
cv-corpus-24.0-2025-12-05/ta/clips/common_voice_ta_40910545.mp3
cv-corpus-24.0-2025-12-05/ta/clips/common_voice_ta_40910546.mp3
cv-corpus-24.0-2025-12-05/ta/clips/common_voice_ta_40910547.mp3
cv-corpus-24.0-2025-12-05/ta/clips/common_voice_ta_40910548.mp3
cv-corpus-24.0-2025-12-05/ta/clips/common_voice_ta_40911469.mp3
cv-corpus-24.0-2025-12-05/ta/clips/common_voice_ta_40911470.mp3
cv-corpus-24.0-2025-12-05/ta/clips/common_voice_ta_40

In [12]:
ls -al ./cv-corpus-24.0-2025-12-05/ta/

total 183944
drwxr-xr-x 3 root  root      4096 Dec 26 10:10 ./
drwxr-xr-x 3 root  root      4096 Dec 26 08:31 ../
-rw-r--r-- 1 10001 10001  8426732 Dec  9 15:18 clip_durations.tsv
drwxr-sr-x 2 10001 10001 12996608 Dec 26 10:13 clips/
-rw-r--r-- 1 10001 10001  5231025 Dec  9 15:20 dev.tsv
-rw-r--r-- 1 10001 10001  2523705 Dec  9 15:20 invalidated.tsv
-rw-r--r-- 1 10001 10001 44389303 Dec  9 15:20 other.tsv
-rw-r--r-- 1 10001 10001  1049619 Dec  9 15:20 reported.tsv
-rw-r--r-- 1 10001 10001  5017808 Dec  9 15:20 test.tsv
-rw-r--r-- 1 10001 10001 19945519 Dec  9 15:20 train.tsv
-rw-r--r-- 1 10001 10001   127867 Dec  9 15:22 unvalidated_sentences.tsv
-rw-r--r-- 1 10001 10001 31338676 Dec  9 15:21 validated_sentences.tsv
-rw-r--r-- 1 10001 10001 57258363 Dec  9 15:20 validated.tsv


In [13]:
import pandas as pd
import os
from datasets import Dataset, DatasetDict, Audio


tsv_path = "/content/cv-corpus-24.0-2025-12-05/ta/validated.tsv"
if os.path.exists(tsv_path):
    print("File exists ✅")
else:
    print("File does not exist ❌")
print(tsv_path)

# Load TSV
train_df = pd.read_csv("/content/cv-corpus-24.0-2025-12-05/ta/train.tsv", sep="\t")
test_df = pd.read_csv("/content/cv-corpus-24.0-2025-12-05/ta/test.tsv", sep="\t")

#[['path', 'sentence']]

# Fix audio paths
data_dir = "/content/cv-corpus-24.0-2025-12-05/ta"
train_df["audio"] = train_df["path"].apply(lambda p: f"{data_dir}/clips/{p}")
test_df["audio"] = test_df["path"].apply(lambda p: f"{data_dir}/clips/{p}")


train_df = train_df[["audio", "sentence"]].rename(columns={"sentence": "text"})
test_df = test_df[["audio", "sentence"]].rename(columns={"sentence": "text"})

# Create HF Dataset
train_hf = Dataset.from_pandas(train_df, preserve_index=False)
test_hf = Dataset.from_pandas(test_df, preserve_index=False)
#ds = ds.cast_column("path", Audio(sampling_rate=16_000))

# Keep only audio + text
#ds = ds.rename_column("sentence", "text")
#ds = ds.select_columns(["path", "text"])

common_voice = DatasetDict({
    "train": train_hf,
    "test": test_hf
})

print(train_hf)
print(test_hf)

File exists ✅
/content/cv-corpus-24.0-2025-12-05/ta/validated.tsv
Dataset({
    features: ['audio', 'text'],
    num_rows: 46113
})
Dataset({
    features: ['audio', 'text'],
    num_rows: 7926
})


Feature Extractor - converts raw audio to log-Mel spectrogram

In [14]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

Tokenizer

In [15]:
from transformers import WhisperTokenizer
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Tamil", task="transcribe")


Verify the tokenizer by encoding and decoding the first audio in the dataset.

In [16]:
input_str = common_voice["train"][0]["text"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 அவரைப் பொதுமக்கள் விடாமல் பின்னாலேயே துரத்திக் கொண்டே ஓடினார்கள்.
Decoded w/ special:    <|startoftranscript|><|ta|><|transcribe|><|notimestamps|>அவரைப் பொதுமக்கள் விடாமல் பின்னாலேயே துரத்திக் கொண்டே ஓடினார்கள்.<|endoftext|>
Decoded w/out special: அவரைப் பொதுமக்கள் விடாமல் பின்னாலேயே துரத்திக் கொண்டே ஓடினார்கள்.
Are equal:             True


In [17]:
from datasets import Audio

#troubleshooting
!pip install torchcodec

print(common_voice["train"].column_names)
print(common_voice["train"][0])
common_voice = common_voice.cast_column(
    "audio",
    Audio(sampling_rate=16000)
)

print(common_voice["train"][0]["audio"]["sampling_rate"])

print(common_voice["train"].column_names)

#====
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

['audio', 'text']
{'audio': '/content/cv-corpus-24.0-2025-12-05/ta/clips/common_voice_ta_26650298.mp3', 'text': 'அவரைப் பொதுமக்கள் விடாமல் பின்னாலேயே துரத்திக் கொண்டே ஓடினார்கள்.'}
16000
['audio', 'text']


Combine extractor and tokenizer into a single class called WhisperProcessor.

In [18]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Tamil", task="transcribe")

function prepares data for the model. then it is mapped to all training examples.

In [19]:
'''
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["text"], truncation=True).input_ids
    return batch
'''
#common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice["train"].column_names)

#alt

import torch

class DataCollatorSpeechSeq2SeqWithPadding:
    def __init__(self, processor):
        self.processor = processor

    def __call__(self, features):
        audio_arrays = [f["audio"]["array"] for f in features]
        batch = self.processor.feature_extractor(
            audio_arrays,
            sampling_rate=16000,
            return_tensors="pt"
        )

        labels = [f["text"] for f in features]
        label_batch = self.processor.tokenizer(
            labels,
            padding=True,
            truncation=True,
            return_tensors="pt"
        )

        batch["labels"] = label_batch["input_ids"]
        return batch


In [25]:
import transformers
from transformers import (
    WhisperForConditionalGeneration,
    WhisperProcessor,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)

model_name = "openai/whisper-base"

processor = WhisperProcessor.from_pretrained(
    model_name,
    language="Tamil",
    task="transcribe"
)

model = WhisperForConditionalGeneration.from_pretrained(model_name)

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

#import inspect
#print(inspect.signature(Seq2SeqTrainingArguments.__init__))



training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-ta",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=2000,
    gradient_checkpointing=True,
    fp16=True,
    eval_strategy="steps",
    save_steps=500,
    eval_steps=500,
    logging_steps=50,
    report_to="none",
)

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
)

/tmp/ipython-input-2133717394.py:46: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
